In [62]:
import torch
import pandas as pd
from operator import itemgetter
import random

In [32]:
# convert code to use cuda in the end 
# torch.cuda.is_available()

In [162]:
def generate_false_samples(entities_dict, sample, head_or_tail, training_data, index_start_of_relation):
    '''
    
    '''
    # 0 for head
    if head_or_tail:
        # generate false head entity
        # to break in case our tail entity has a relation with every other entity 
        counter = 0
        while(True):
            unique = True
            random_entity = random.choice(list(entities_dict.values()))
            for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
                try:
                    if torch.equal(random_entity, training_data[i][0]) and torch.equal(sample[2], training_data[i][2]):
                        unique = False
                        break
                except:
                    print(random_entity)
                    print(index_start_of_relation)
#                     print("item", item)
                    print("training data", len(training_data))
                counter += 1
            if unique:
                break
            if counter == 10000:
                print("problable mistake")
        sample.append(random_entity)
        sample.append(sample[2])
    # 1 for tail
    else:
        # generate false tail entity
        counter = 0
        while(True):
            unique = True
            random_entity = random.choice(list(entities_dict.values()))
            for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
                if torch.equal(sample[0], training_data[i][0]) and torch.equal(random_entity, training_data[i][2]):
                    unique = False
                    break
                counter += 1
            if unique or counter == 10000:
                break
        sample.append(sample[0]) 
        sample.append(random_entity)
    return sample

In [4]:
entities_temp_list = pd.read_csv('./data/wn18rr/entities.dict', header=None, sep='\t')[1]

In [5]:
# entities_temp_list
# torch.eye(len(entities_temp_list))

In [6]:
entities_matrix = torch.eye(len(entities_temp_list))
entities_vectors = torch.split(entities_matrix, 1)

In [7]:
# entities_vectors
entities = dict()

In [8]:
for i in range(len(entities_temp_list)):
    entities[entities_temp_list[i]] = torch.reshape(entities_vectors[i][0], (-1, 1))
    entities[entities_temp_list[i]].requires_grad=True

In [9]:
# entities[260881].shape

In [10]:
relation_temp_list = pd.read_csv('./data/wn18rr/relations.dict', header=None, sep='\t')[1]

In [43]:
relation_temp_list[0]

'_hypernym'

In [14]:
relation_matrix = torch.eye(len(relation_temp_list))
relation_vectors = torch.split(relation_matrix, 1)

In [18]:
# relation_vectors[0][0]

In [19]:
relations = dict()

In [20]:
for i in range(len(relation_temp_list)):
    relations[relation_temp_list[i]] = torch.reshape(relation_vectors[i][0], (-1, 1))
    relations[relation_temp_list[i]].requires_grad=True
#     print(relations[relation_temp_list[i]])
# print(relations)

In [152]:
read_training_data = pd.read_csv('./data/wn18rr/train.txt', header=None, sep='\t')

In [153]:
training_data = read_training_data.values.tolist()
training_data = sorted(training_data, key=itemgetter(1))

In [154]:
# training_data

In [155]:
# we get the embeddings for (h, r, t# training_data)
for i in range(len(training_data)):
    training_data[i][0] = entities[training_data[i][0]]
    training_data[i][1] = relations[training_data[i][1]]
    training_data[i][2] = entities[training_data[i][2]]
# training_data

In [161]:
# generate_false_samples(entities_dict, sample, head_or_tail, training_data, index_start_of_relation)
# looks pretty inefficient

# try creating a dict with the relation (adding _head or _tail to the end)
# as the key and the values are a list with every head or tail that is associated with this relation

index_start_of_relation = 0
for i  in range(len(training_data)):
    print(i)
    if i > 0 and torch.equal(training_data[i][1], training_data[i-1][1]):
        index_start_of_relation = i
    head_or_tail = random.randint(0, 1)
    training_data[i] = generate_false_samples(entities, training_data[i], head_or_tail, training_data, index_start_of_relation)

In [157]:
training_data[0]

[tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]], requires_grad=True),
 tensor([[0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]], requires_grad=True),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]], requires_grad=True),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]], requires_grad=True),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]], requires_grad=True)]

In [158]:
len(training_data)

86835